In [4]:
# given string (mention) form text, we assign the string to the wiki page (entity) whose name perfectly matches
# of course this is nothing but a demonstrative example
# input: list of candidates from each text
# output: mention-entity pair

import os, json, pprint, pickle
from urllib.parse import unquote
root = '/data/cm/'

In [5]:
candidates = pickle.load(open(root+'output/candidate_selection/matched_wiki_from_NER.pickle', 'rb'))
# candidate strings from the documents (extracted from NER) that we want to link to Wiki

In [6]:
commonness = pickle.load(open(root+'data/wiki/de/commonness.pickle', 'rb'))

In [17]:
commonness['Hafenstadt%20Haifa']

KeyError: 'Hafenstadt%20Haifa'

In [8]:
pprint.pprint(candidates['9620162']['9633717_9625984__511_9625989'][1])

{'b': 159,
 'e': 164,
 'entities': {'ATP%20Challenger%20Tunis',
              'Erzbistum%20Tunis',
              'Flughafen%20Tunis',
              'Geschichte%20Tunesiens%23Osmanen%20%281574%20bis%201790%20bzw.%201881%29',
              'Gouvernement%20Tunis',
              'Grand%20Prix%20de%20Tunisie',
              'Gro%C3%9Fer%20Preis%20von%20Tunesien',
              'Liste%20der%20Beys%20von%20Tunis',
              'Regentschaft%20Tunis',
              'Stadion%20des%207.%20November',
              'Tunesien',
              'Tunesienfeldzug',
              'Tunis',
              'Tunis%20%28Fautmolo%29',
              'Tunis%20%28Gouvernorat%29',
              'Tunis%20Open%202009',
              'Tunis%20Open%202010%23Doppel',
              'Tunis%23Geschichte',
              'Tunisfeldzug'},
 'text': 'Tunis'}


In [9]:
matches = {k: {} for k in candidates.keys()}
for journal, pages in candidates.items():
    print('Processing', journal)
    
    for page, entries in pages.items():
        
        if page not in matches[journal].keys():
            matches[journal][page] = []
        
        for entry in entries:
            target = entry['text'] # string from document to disambiguate
            unquoted_entities = [unquote(e) for e in entry['entities']]
            # compute commonness for each candidate entities
            local_comm = []
            for e in entry['entities']:
                try:
                    local_comm.append((e, commonness[e][target]))
                except KeyError:
                    pass
            
            local_comm = sorted(local_comm, key=lambda x:x[1], reverse=True)
            #print(target, local_comm[:4])
            #print()
            
            # assign the entity with the highest commonness for that mention
            if local_comm != []:
                
                match = {
                    'text': entry['text'],
                    'b': entry['b'],
                    'e': entry['e'],
                    'match': 'https://de.wikipedia.org/wiki/'+local_comm[0][0]
                }
                matches[journal][page].append(match)
                
    

Processing 9620162
Processing 3139318
Processing 2566634


In [10]:
# provide some examples of the entity in context
# 3139318 3133277_3133281__177_3133480 81 92 Paul Bekker
# 3139318 3133277_3133281__180_3133483 481 489 Bayreuth
# 9620162 9632912_9625712__234_9625713 5288 5301 Theodor Heuss
# 9620162 9632912_9625712__234_9625713 5408 5422 Heinrich Lübke

journal = '9620162'
page = '9632912_9625712__234_9625713'
b = int('5288')
e = int('5301')
url = 'https://de.wikipedia.org/wiki/Theodor%20Heuss'

text = open(root+'data/pages/'+journal+'/'+page+'.txt', 'r', encoding='utf-8').read()
entity = text[b:e]
text_span = text[b-50:e+50]
print(text_span)
print(entity)
print(url)

werden). III Am 15. September gibt der Protestant Theodor Heuss sein seit 10 Jahren hervorragend geführtes Bunlde
Theodor Heuss
https://de.wikipedia.org/wiki/Theodor%20Heuss


In [61]:
# show random matches with text span
import random
r = random.randint(0, 18142)
#print(r)

i = 0
for journal, pages in matches.items():
    for page, values in pages.items():
        for match in values:
            i += 1
            if i == r:
                text = open(root+'data/pages/'+journal+'/'+page+'.txt', 'r', encoding='utf-8').read()
                b = int(match['b'])
                e = int(match['e'])
                url = match['match']
                text_span = text[b-80:e+80]
                entity_name = match['text']
                print('Found mention ***', entity_name, '*** in text:')
                print()
                print(text_span)
                print()
                print('Begin:', b, 'End:', e)
                print('url:', url)
                break

Found mention *** jüdischen *** in text:

lichen, Inhalt seines Werkes wird. Friedlaender betonte überzeugend den starken jüdischen Kern im Wesen des deutschen Literaten Lion Feuchtwanger, mit dessen Tod einer d

Begin: 10262 End: 10271
url: https://de.wikipedia.org/wiki/Judentum
